In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    print("Not running in Colab, skipping Google Drive mount.")

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
from PIL import Image
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import numpy as np

# ======================
# CONFIG
# ======================
BASE      = Path("/content/drive/MyDrive/FakeAVCeleb")
DATA_DIR  = BASE/"frames"                                # frames/{sample_id}/frame_*.jpg
TRAIN_CSV = BASE/"train_fixed_fullschema_hashsplit.csv"
TEST_CSV  = BASE/"test_fixed_fullschema_hashsplit.csv"
MODEL_PATH = "/content/drive/MyDrive/FakeAVCeleb/mesonet.pth"
OUT_DIR   = BASE/"fusion_video_meso"                     # embeddings
OUT_CSV   = BASE/"fusion_video_meso_index.csv"
OUT_DIR.mkdir(parents=True, exist_ok=True)

IMG_SIZE = 256
BATCH_SIZE = 16
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ======================
# MODEL
# ======================
class Meso4(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=3, padding=1), nn.BatchNorm2d(8), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(8, 8, kernel_size=3, padding=1), nn.BatchNorm2d(8), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(8, 16, kernel_size=3, padding=1), nn.BatchNorm2d(16), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 16, kernel_size=3, padding=1), nn.BatchNorm2d(16), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.fc1 = nn.Linear(16 * (IMG_SIZE // 16) * (IMG_SIZE // 16), 16)
        self.fc2 = nn.Linear(16, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x, return_feat=False):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.size(0), -1)
        feat = torch.relu(self.fc1(x))  # [B,16]
        if return_feat:
            return feat
        feat = self.dropout(feat)
        return self.fc2(feat)

# reload model
meso = Meso4(num_classes=2).to(DEVICE)
meso.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
meso.eval()

# ======================
# TRANSFORM
# ======================
val_test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# ======================
# EXPORT LOOP
# ======================
def export_split(df: pd.DataFrame, split: str):
    rows = []
    for _, row in tqdm(df.iterrows(), total=len(df), desc=f"Export {split} emb"):
        sid = str(row['sample_id'])
        label = int(row['video_fake'])
        folder = DATA_DIR/sid
        frames = sorted(folder.glob("frame_*.jpg"))
        if not frames:
            continue

        feats = []
        with torch.no_grad():
            for f in frames:
                img = Image.open(f).convert('RGB')
                img = val_test_transform(img).unsqueeze(0).to(DEVICE)  # [1,3,H,W]
                feat = meso(img, return_feat=True).cpu().numpy()[0]
                feats.append(feat)

        emb = np.mean(feats, axis=0).astype(np.float32)  # [16]
        path = OUT_DIR/f"{sid}_meso16.npy"
        np.save(path, emb)
        rows.append({"sample_id": sid,
                     "meso_emb_path": str(path),
                     "label": label,
                     "split": split})
    return rows

train_df = pd.read_csv(TRAIN_CSV)[["sample_id","video_fake"]]
test_df  = pd.read_csv(TEST_CSV)[["sample_id","video_fake"]]

all_rows = []
all_rows += export_split(train_df, "train")
all_rows += export_split(test_df, "test")

pd.DataFrame(all_rows).to_csv(OUT_CSV, index=False)
print(f"✓ Saved {len(all_rows)} Meso4 video embeddings → {OUT_CSV}")


Export test emb: 100%|██████████| 407/407 [24:39<00:00,  3.63s/it]

✓ Saved 2000 Meso4 video embeddings → /content/drive/MyDrive/FakeAVCeleb/fusion_video_meso_index.csv


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.models import vgg19, VGG19_Weights
import torch.nn.functional as F
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import numpy as np
import librosa

# ======================
# CONFIG
# ======================
BASE_DIR   = Path("/content/drive/MyDrive/FakeAVCeleb")
TRAIN_CSV  = BASE_DIR/"train_fixed_fullschema_hashsplit.csv"
TEST_CSV   = BASE_DIR/"test_fixed_fullschema_hashsplit.csv"
AUDIO_ROOT = BASE_DIR/"audio_wav"

BEST_PATH  = BASE_DIR/"audio_vgg_3ch_best_robust.pth"
OUT_DIR    = BASE_DIR/"fusion_audio_vgg"
OUT_CSV    = BASE_DIR/"fusion_audio_vgg_index.csv"
OUT_DIR.mkdir(parents=True, exist_ok=True)

DEVICE     = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SAMPLE_RATE= 16000
N_MELS     = 128
N_MFCC     = 20
DURA_SEC   = 4.0
TARGET_LEN = int(SAMPLE_RATE * DURA_SEC)

# ======================
# FEATURE EXTRACTION
# ======================
def _pad_or_trim(y, target_len):
    if len(y) > target_len: return y[:target_len]
    if len(y) < target_len: return np.pad(y, (0, target_len - len(y)))
    return y

def extract_audio_features(path: Path) -> torch.Tensor:
    y, sr = librosa.load(path, sr=SAMPLE_RATE)
    y = _pad_or_trim(y, TARGET_LEN)

    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=N_MELS)
    mel_db = librosa.power_to_db(mel, ref=np.max)
    mel_norm = (mel_db - mel_db.min()) / (mel_db.max() - mel_db.min() + 1e-6)

    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=N_MFCC)
    mfcc_norm = (mfcc - mfcc.min()) / (mfcc.max() - mfcc.min() + 1e-6)

    mfcc_delta = librosa.feature.delta(mfcc)
    mfcc_delta_norm = (mfcc_delta - mfcc_delta.min()) / (mfcc_delta.max() - mfcc_delta.min() + 1e-6)

    mel_t   = torch.tensor(mel_norm, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
    mfcc_t  = torch.tensor(mfcc_norm, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
    delta_t = torch.tensor(mfcc_delta_norm, dtype=torch.float32).unsqueeze(0).unsqueeze(0)

    mfcc_r  = F.interpolate(mfcc_t,  size=(128, mel_t.shape[-1]), mode="bilinear", align_corners=False)
    delta_r = F.interpolate(delta_t, size=(128, mel_t.shape[-1]), mode="bilinear", align_corners=False)

    feat = torch.cat([mel_t.squeeze(0), mfcc_r.squeeze(0), delta_r.squeeze(0)], dim=0)
    feat = F.interpolate(feat.unsqueeze(0), size=(128,128), mode="bilinear", align_corners=False).squeeze(0)
    return feat  # [3,128,128]

# ======================
# DATASET
# ======================
class AudioDataset(Dataset):
    def __init__(self, df, audio_root: Path):
        self.df = df.reset_index(drop=True)
        self.audio_root = audio_root

    def __len__(self): return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        sid = str(row["sample_id"])
        label = int(row["audio_fake"])
        feat = extract_audio_features(self.audio_root/f"{sid}.wav")
        return feat, label, sid

# ======================
# MODEL (with feature hook)
# ======================
class AudioVGGFeatures(nn.Module):
    def __init__(self):
        super().__init__()
        self.vgg = vgg19(weights=VGG19_Weights.DEFAULT)
        self.vgg.features[0] = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.vgg.classifier[-1] = nn.Linear(4096, 1)

    def forward(self, x, return_feat=False):
        y = self.vgg.features(x)
        y = self.vgg.avgpool(y)
        y = torch.flatten(y, 1)
        feat = self.vgg.classifier[:5](y)     # [B,4096]
        if return_feat:
            return feat
        logit = self.vgg.classifier[5:](feat).squeeze(1)
        return logit

# ======================
# LOAD MODEL
# ======================
model = AudioVGGFeatures().to(DEVICE)
model.load_state_dict(torch.load(BEST_PATH, map_location=DEVICE))
model.eval()
print(f"Loaded weights ← {BEST_PATH}")

# ======================
# EXPORT LOOP
# ======================
def export_split(df: pd.DataFrame, split: str):
    ds = AudioDataset(df, AUDIO_ROOT)
    dl = DataLoader(ds, batch_size=8, shuffle=False, num_workers=2)
    rows = []

    with torch.no_grad():
        for xb, yb, sids in tqdm(dl, desc=f"Export {split} emb"):
            xb = xb.to(DEVICE)
            feats = model(xb, return_feat=True).cpu().numpy()  # [B,4096]
            for sid, emb, lbl in zip(sids, feats, yb.numpy().astype(int)):
                path = OUT_DIR/f"{sid}_vgg4096.npy"
                np.save(path, emb.astype(np.float32))
                rows.append({
                    "sample_id": sid,
                    "audio_vgg_emb_path": str(path),
                    "label": lbl,
                    "split": split
                })
    return rows

train_df = pd.read_csv(TRAIN_CSV)[["sample_id","audio_fake"]]
test_df  = pd.read_csv(TEST_CSV)[["sample_id","audio_fake"]]

all_rows = []
all_rows += export_split(train_df, "train")
all_rows += export_split(test_df, "test")

pd.DataFrame(all_rows).to_csv(OUT_CSV, index=False)
print(f"✓ Saved {len(all_rows)} Audio VGG19 embeddings → {OUT_CSV}")


Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


100%|██████████| 548M/548M [00:07<00:00, 74.7MB/s]


Loaded weights ← /content/drive/MyDrive/FakeAVCeleb/audio_vgg_3ch_best_robust.pth


Export test emb: 100%|██████████| 51/51 [02:50<00:00,  3.33s/it]

✓ Saved 2000 Audio VGG19 embeddings → /content/drive/MyDrive/FakeAVCeleb/fusion_audio_vgg_index.csv


In [ ]:
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd, numpy as np
from pathlib import Path
import librosa
from tqdm import tqdm

# --- CONFIG ---
DEVICE        = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BASE_DIR      = Path('/content/drive/MyDrive/FakeAVCeleb')
TRAIN_CSV     = BASE_DIR/'train_fixed_fullschema_hashsplit.csv'
TEST_CSV      = BASE_DIR/'test_fixed_fullschema_hashsplit.csv'
AUDIO_ROOT    = BASE_DIR/'audio_wav'
CACHE_DIR     = BASE_DIR/'cache/audio_cnn_mel'
BEST_PATH     = BASE_DIR/'audio_cnn_best.pth'

OUT_DIR       = BASE_DIR/'fusion_audio_cnn'
OUT_CSV       = BASE_DIR/'fusion_audio_cnn_index.csv'
OUT_DIR.mkdir(parents=True, exist_ok=True)

SAMPLE_RATE   = 16000
N_MELS        = 128
DURA_SEC      = 4.0
TARGET_LEN    = int(SAMPLE_RATE * DURA_SEC)

# --- Mel-Spectrogram extraction ---
def audio_to_mel(path: Path) -> torch.Tensor:
    y, sr = librosa.load(path, sr=SAMPLE_RATE)
    if len(y) > TARGET_LEN:
        y = y[:TARGET_LEN]
    else:
        y = np.pad(y, (0, TARGET_LEN - len(y)))
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=N_MELS)
    mel_db = librosa.power_to_db(mel, ref=np.max)
    mel_norm = (mel_db - mel_db.min()) / (mel_db.max() - mel_db.min() + 1e-6)
    return torch.tensor(mel_norm, dtype=torch.float32)  # [128, T]

# --- Dataset ---
class MelDataset(Dataset):
    def __init__(self, df: pd.DataFrame, split: str):
        self.df = df.reset_index(drop=True)
        self.split = split

    def __len__(self): return len(self.df)

    def __getitem__(self, i):
        row = self.df.iloc[i]
        sid = str(row['sample_id'])
        label = int(row['audio_fake'])
        pt_path = CACHE_DIR/f"{sid}.pt"

        if pt_path.exists():
            mel = torch.load(pt_path)
        else:
            mel = audio_to_mel(AUDIO_ROOT/f"{sid}.wav")
            torch.save(mel, pt_path)

        img = mel.unsqueeze(0)   # [1,128,T]
        return img, label, sid

# --- Model (feature extractor) ---
class AudioCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64), nn.ReLU(),
            nn.AdaptiveAvgPool2d((4,4)),
            nn.Flatten(),
            nn.Linear(64*4*4, 128), nn.ReLU(), nn.Dropout(0.5)
        )
        self.head = nn.Linear(128, 1)

    def forward(self, x, return_feat=False):
        feat = self.cnn(x)          # [B,128]
        if return_feat:
            return feat
        return self.head(feat).squeeze(1)

# --- Load model ---
model = AudioCNN().to(DEVICE)
model.load_state_dict(torch.load(BEST_PATH, map_location=DEVICE))
model.eval()
print(f"Loaded best weights ← {BEST_PATH.name}")

# --- Export embeddings ---
def export_split(df: pd.DataFrame, split: str):
    ds = MelDataset(df, split)
    dl = DataLoader(ds, batch_size=8, shuffle=False, num_workers=2)
    rows = []
    with torch.no_grad():
        for xb, yb, sids in tqdm(dl, desc=f"Export {split} emb"):
            xb = xb.to(DEVICE)
            feats = model(xb, return_feat=True).cpu().numpy()  # [B,128]
            for sid, emb, lbl in zip(sids, feats, yb.numpy().astype(int)):
                path = OUT_DIR/f"{sid}_cnn128.npy"
                np.save(path, emb.astype(np.float32))
                rows.append({
                    "sample_id": sid,
                    "audio_cnn_emb_path": str(path),
                    "label": lbl,
                    "split": split
                })
    return rows

train_df = pd.read_csv(TRAIN_CSV)[['sample_id','audio_fake']]
test_df  = pd.read_csv(TEST_CSV)[['sample_id','audio_fake']]

all_rows = []
all_rows += export_split(train_df, "train")
all_rows += export_split(test_df, "test")

pd.DataFrame(all_rows).to_csv(OUT_CSV, index=False)
print(f"✓ Saved {len(all_rows)} Audio CNN embeddings → {OUT_CSV}")


Loaded best weights ← audio_cnn_best.pth


Export test emb: 100%|██████████| 51/51 [01:01<00:00,  1.21s/it]

✓ Saved 2000 Audio CNN embeddings → /content/drive/MyDrive/FakeAVCeleb/fusion_audio_cnn_index.csv


In [ ]:
import pandas as pd

# Load all indexes
resnet_df = pd.read_csv("/content/drive/MyDrive/FakeAVCeleb/fusion_video_emb_index.csv")
meso_df   = pd.read_csv("/content/drive/MyDrive/FakeAVCeleb/fusion_video_meso_index.csv")
vgg_df    = pd.read_csv("/content/drive/MyDrive/FakeAVCeleb/fusion_audio_vgg_index.csv")
cnn_df    = pd.read_csv("/content/drive/MyDrive/FakeAVCeleb/fusion_audio_cnn_index.csv")

# Merge step by step
df = resnet_df.merge(meso_df[["sample_id","meso_emb_path"]], on="sample_id")
df = df.merge(vgg_df[["sample_id","audio_vgg_emb_path"]], on="sample_id")
df = df.merge(cnn_df[["sample_id","audio_cnn_emb_path"]], on="sample_id")

# keep label/split from resnet_df (all aligned by hash split)
print(df.head())
print("Final merged shape:", df.shape)

df.to_csv("/content/drive/MyDrive/FakeAVCeleb/fusion_master_index.csv", index=False)
print("✓ Saved merged index → fusion_master_index.csv")


      sample_id                                     video_emb_path  label  \
0  sample_00991  /content/drive/MyDrive/FakeAVCeleb/fusion_vide...      0   
1  sample_00985  /content/drive/MyDrive/FakeAVCeleb/fusion_vide...      0   
2  sample_01173  /content/drive/MyDrive/FakeAVCeleb/fusion_vide...      1   
3  sample_01044  /content/drive/MyDrive/FakeAVCeleb/fusion_vide...      1   
4  sample_00838  /content/drive/MyDrive/FakeAVCeleb/fusion_vide...      0   

   split                                      meso_emb_path  \
0  train  /content/drive/MyDrive/FakeAVCeleb/fusion_vide...   
1  train  /content/drive/MyDrive/FakeAVCeleb/fusion_vide...   
2  train  /content/drive/MyDrive/FakeAVCeleb/fusion_vide...   
3  train  /content/drive/MyDrive/FakeAVCeleb/fusion_vide...   
4  train  /content/drive/MyDrive/FakeAVCeleb/fusion_vide...   

                                  audio_vgg_emb_path  \
0  /content/drive/MyDrive/FakeAVCeleb/fusion_audi...   
1  /content/drive/MyDrive/FakeAVCeleb/fusion_a

In [ ]:
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np, pandas as pd
from pathlib import Path
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
INDEX_CSV = "/content/drive/MyDrive/FakeAVCeleb/fusion_master_index.csv"
BATCH_SIZE = 32
EPOCHS = 10
LR = 1e-4
WD = 1e-5

# === Dataset ===
class FusionDataset(Dataset):
    def __init__(self, df):
        self.df = df.reset_index(drop=True)
    def __len__(self): return len(self.df)
    def __getitem__(self, i):
        row = self.df.iloc[i]
        # load each embedding
        v_resnet = np.load(row["video_emb_path"])
        v_meso   = np.load(row["meso_emb_path"])
        a_vgg    = np.load(row["audio_vgg_emb_path"])
        a_cnn    = np.load(row["audio_cnn_emb_path"])
        # concat
        feat = np.concatenate([v_resnet, v_meso, a_vgg, a_cnn]).astype(np.float32)
        label = np.array(row["label"], dtype=np.float32)
        return torch.tensor(feat), torch.tensor(label)

# === Model ===
class FusionMLP(nn.Module):
    def __init__(self, in_dim=4752):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 512), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(512, 128), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(128, 1)
        )
    def forward(self, x): return self.net(x).squeeze(1)

# === Data ===
df = pd.read_csv(INDEX_CSV)
train_df = df[df["split"]=="train"]
test_df  = df[df["split"]=="test"]

train_dl = DataLoader(FusionDataset(train_df), batch_size=BATCH_SIZE, shuffle=True)
test_dl  = DataLoader(FusionDataset(test_df),  batch_size=BATCH_SIZE, shuffle=False)

# === Train ===
model = FusionMLP().to(DEVICE)
crit = nn.BCEWithLogitsLoss()
opt = optim.AdamW(model.parameters(), lr=LR, weight_decay=WD)

best_acc = 0
for epoch in range(1, EPOCHS+1):
    model.train(); total_loss=0; preds=[]; labels=[]
    for xb,yb in tqdm(train_dl, desc=f"Epoch {epoch} Train"):
        xb,yb = xb.to(DEVICE), yb.to(DEVICE)
        opt.zero_grad()
        out = model(xb)
        loss = crit(out, yb)
        loss.backward(); opt.step()
        total_loss += loss.item()
        preds += (torch.sigmoid(out)>0.5).detach().cpu().numpy().tolist()
        labels += yb.cpu().numpy().tolist()
    acc = accuracy_score(labels, preds)
    print(f"→ Train Loss {total_loss/len(train_dl):.4f} Acc {acc:.3f}")

    # Eval
    model.eval(); preds=[]; labels=[]
    with torch.no_grad():
        for xb,yb in test_dl:
            xb,yb = xb.to(DEVICE), yb.to(DEVICE)
            out = model(xb)
            preds += (torch.sigmoid(out)>0.5).cpu().numpy().tolist()
            labels += yb.cpu().numpy().tolist()
    acc = accuracy_score(labels, preds)
    print(f"→ Test Acc {acc:.3f}")
    if acc>best_acc:
        best_acc=acc
        torch.save(model.state_dict(), "/content/drive/MyDrive/FakeAVCeleb/fusion_mlp_best.pth")
        print("✓ Saved new best fusion model")

# Final classification report
model.load_state_dict(torch.load("/content/drive/MyDrive/FakeAVCeleb/fusion_mlp_best.pth"))
model.eval(); preds=[]; labels=[]
with torch.no_grad():
    for xb,yb in test_dl:
        xb,yb = xb.to(DEVICE), yb.to(DEVICE)
        out = model(xb)
        preds += (torch.sigmoid(out)>0.5).cpu().numpy().tolist()
        labels += yb.cpu().numpy().tolist()
print(classification_report(labels, preds, target_names=["Real","Fake"]))


Epoch 1 Train: 100%|██████████| 50/50 [22:46<00:00, 27.33s/it]


→ Train Loss 0.3814 Acc 0.892
→ Test Acc 0.956
✓ Saved new best fusion model


Epoch 2 Train: 100%|██████████| 50/50 [00:20<00:00,  2.46it/s]


→ Train Loss 0.1456 Acc 0.952
→ Test Acc 0.953


Epoch 3 Train: 100%|██████████| 50/50 [00:17<00:00,  2.81it/s]


...

→ Train Loss 0.0971 Acc 0.969
→ Test Acc 0.966


Epoch 9 Train: 100%|██████████| 50/50 [00:19<00:00,  2.57it/s]


→ Train Loss 0.0933 Acc 0.969
→ Test Acc 0.968
✓ Saved new best fusion model


Epoch 10 Train: 100%|██████████| 50/50 [00:18<00:00,  2.66it/s]


→ Train Loss 0.0955 Acc 0.968
→ Test Acc 0.971
✓ Saved new best fusion model
              precision    recall  f1-score   support

        Real       0.97      0.97      0.97       200
        Fake       0.97      0.97      0.97       207

    accuracy                           0.97       407
   macro avg       0.97      0.97      0.97       407
weighted avg       0.97      0.97      0.97       407

